# Жанровая классификация аудио

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import keras
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import glob
import torch
import torchsummary as ts
import keras

import json

from functools import reduce
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss

from keras.backend import clear_session
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger, BackupAndRestore

from utils import label2vec

%load_ext autoreload
%autoreload 2

In [ ]:
print(tf.config.list_physical_devices())
print(f'MPS is available: {torch.backends.mps.is_available()}')
print(f'MPS is built: {torch.backends.mps.is_built()}')

## Фильтрация метаданных

In [ ]:
DATA_DIR = './data/fma_small'
METADATA_DIR = './data/fma_metadata/'

mp3_files = glob.glob(DATA_DIR + '/*/*.mp3')
mp3_names = list(map(lambda f: np.int64(f.split('/')[-1].split('.')[0]), mp3_files))

raw_tracks = pd.read_csv(METADATA_DIR + 'raw_tracks.csv')
tracks = raw_tracks[raw_tracks['track_id'].isin(mp3_names)]

## Сбор признаков, полученных с помощью `librosa`

In [ ]:
features_df = pd.read_csv(METADATA_DIR + 'features.csv', index_col=0, header=[0, 1, 2])
features_df = features_df[features_df.index.isin(mp3_names)]

features = np.unique(list(map(lambda x: x[0], list(features_df.columns))))

print(f"Features available: {features}")
print(f"Total: {len(features)}")

features_df

## Отбор признаков

Рассмотрим всю имеющуюся информацию о треках

In [ ]:
tracks.columns

Оценим число непустых значений тегов

In [ ]:
tracks['tags'].map(lambda x: None if x == '[]' else x).notnull().value_counts()

Подсчитаем число уникальных тегов

In [ ]:
unique_tags = reduce(lambda tags, l: tags.union(eval(l)), tracks['tags'], set())
print(len(unique_tags))

Оставим предположительно полезную информацию из набора данных. Убедимся
в её необходимости позже.

In [ ]:
to_keep = [
  'track_id', "album_id", "artist_id", "track_duration", 
  "track_genres", "track_instrumental", "track_interest", "track_listens",
]

filtered_tracks = tracks[to_keep]
filtered_tracks

Преобразуем время в секунды

In [ ]:
def duration_to_int(t):
  splitted = t.split(":")
  
  return int(splitted[0]) * 60 + int(splitted[1])

filtered_tracks.loc[:,'track_duration'] = filtered_tracks.track_duration.apply(duration_to_int)
filtered_tracks

Узнаем количество жанров для треков

In [ ]:
genres = filtered_tracks['track_genres'].map(lambda x: json.loads(x.replace("'", "\"")))
genre_ids = genres.map(lambda x: list(map(lambda y: y['genre_id'], x)))
genre_ids.map(lambda x: len(x)).value_counts()

Определим базовые жанры для каждого трека

In [ ]:
all_genres = pd.read_csv(METADATA_DIR + 'genres.csv')

base_genres = genre_ids.map(lambda x: all_genres[all_genres.genre_id == int(x[0])].iloc[0].top_level)

filtered_tracks['track_genres'] = base_genres
filtered_tracks

In [ ]:
base_genres.value_counts()

Получили 8 сбалансированных классов

In [ ]:
def display_corr(df):
  corr = df.corr()
  cmap = sns.diverging_palette(230, 20, as_cmap=True)
  mask = np.triu(np.ones_like(corr, dtype=bool))
  sns.heatmap(corr, mask=mask, cmap=cmap)
  
display_corr(filtered_tracks)

Жанр трека очень плохо коррелирует с его длительностью, поэтому исключим
этот признак из рассмотрения

In [ ]:
filtered_tracks = filtered_tracks.drop('track_duration', axis=1)

Теперь добавим значения, предпосчитанные с помощью `librosa`

In [ ]:
merged = features_df.merge(filtered_tracks, how='inner', on='track_id')

display_corr(merged)

Конечно, признаков слишком много. Из всех возьмем признаки с наибольшей по
модулю корреляцией.

Для этого отсортируем признаки по степени корреляции

In [ ]:
correlation = merged.corr()

genres_corr = correlation['track_genres'].sort_values(key=lambda x: np.abs(x), ascending=False)
genres_corr

Изобразим распределение значений корреляции

In [ ]:
sns.histplot(genres_corr)

Видно, что наибольшее число признаков имеют почти нулевую корреляцию.
В связи с этим выберем наиболее информативные из них

In [ ]:
boundary = 0.2

selected = merged[genres_corr[abs(genres_corr) > boundary].reset_index()['index']]
selected.set_index('track_id', inplace=True)

Кроме того, удалим сильно коррелирующие друг с другом нецелевые признаки,
оставив среди таких пар те, что больше коррелируют с целевым

In [ ]:
c = selected.corr()

to_be_excluded = set()

boundary = 0.9

for i in c:
  for j in c:
    if abs(c[i][j]) > boundary and i != j and i != 'track_genres' and j != 'track_genres':
      least_informative = i if c['track_genres'][i] < c['track_genres'][j] else j
      to_be_excluded.add(least_informative)
      
to_be_excluded

In [ ]:
selected = selected.drop(to_be_excluded, axis=1)

Перекодируем метки классов

In [ ]:
genre_le = LabelEncoder()

selected.track_genres = genre_le.fit_transform(selected.track_genres)
selected

In [ ]:
selected.columns = selected.columns.map(str)

In [ ]:
for column in selected.columns:
  if column == 'track_genres':
    continue
  selected[column] = StandardScaler().fit_transform(selected[column].to_numpy().reshape(-1, 1))

Убедимся, что `StandardScaler` отработал корректно

In [ ]:
selected.describe()

Разделим данные по принципу `train/test/split`

In [ ]:
x = selected.drop('track_genres', axis=1).to_numpy()
y = selected['track_genres'].to_numpy()

test_size = 0.2
valid_size = 0.1

X_train, X_test, y_train, y_test = \
    train_test_split(x, y, test_size=0.2, random_state=69, stratify=y)
    
X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=valid_size / (1 - test_size),
                     random_state=69, stratify=y_train)
    
n_classes = np.max(y) + 1

## Сравнение моделей

In [ ]:
models = []

### K-Nearest Neighbours

In [ ]:
n_classes = np.max(y) + 1
list_of_neighbours = list(map(int, range(1, 300, 5)))

Опишем функцию, которая будет отображать результаты экспериментов

In [ ]:
def plot_score(n, scores, names):
    d = {names: n, 'score': scores}
    df = pd.DataFrame(d)

    sns.set(style='darkgrid')
    sns.lineplot(x=names, y='score', data=df)

Вычислим значения `accuracy` для моделей с разным числом соседей

In [ ]:
best_n = -1
best_score = -1
scores = []
for n in tqdm(list_of_neighbours):
    knn = KNeighborsClassifier(p=1, n_neighbors=n)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    score = knn.score(X_test, y_test)
    
    if score > best_score:
        best_score = score
        best_n = n
    scores.append(score)

plot_score(list_of_neighbours, scores, 'neighbors')
print(f'Лучшая модель: {best_n} соседей, точность: {best_score}')

Конечно, такой метод оценки качества модели не является надежным, лучше воспользоваться
оценкой методом кросс-валидации — `cross_val_score`.

В дальнейшем будем использовать именно этот метод.

In [ ]:
best_n = -1
best_score = -1
scores = []

for n in tqdm(list_of_neighbours):
    knn = KNeighborsClassifier(p=2, n_neighbors=n)
    
    score = cross_val_score(knn, x, y, cv=5).mean()
    
    if score > best_score:
        best_score = score
        best_n = n
    scores.append(score)

plot_score(list_of_neighbours, scores, 'neighbors')
print(f'Лучшая модель: {best_n} соседей, точность: {best_score}')

models.append((KNeighborsClassifier(p=2, n_neighbors=best_n), 'sklearn'))

In [ ]:
scores = []
for n in tqdm(list_of_neighbours):
    knn = KNeighborsClassifier(p=2, n_neighbors=n)
    knn.fit(X_train, y_train)
    probs = knn.predict_proba(X_test)
    
    loss = log_loss(y_test, probs)
    scores.append(loss)

plot_score(list_of_neighbours, scores, 'neighbors')

### SVC

#### C-svc

#### форма функции решения: one vs one

In [ ]:
from sklearn import svm


scores = []
ps = list(map(int, range(1, 10, 1)))
for p in tqdm(ps):
    svc = svm.SVC(degree=p, decision_function_shape='ovo')
    svc.fit(X_train, y_train)
    scores.append(svc.score(X_test, y_test))

plot_score(ps, scores, 'degree')

In [ ]:
scores = []
Cs = list(map(int, range(1, 100, 1)))

for c in tqdm(Cs):
    svc = svm.SVC(C=c, decision_function_shape='ovo')
    svc.fit(X_train, y_train)
    scores.append(svc.score(X_test, y_test))

plot_score(Cs, scores, 'C-argument')

In [ ]:
scores =[]
for c in tqdm(Cs):
    svc = svm.SVC(C=c, decision_function_shape='ovo')
    cvs = cross_val_score(svc, x, y, cv=5)
    scores.append(cvs.mean())

plot_score(Cs, scores, 'C-argument')

In [ ]:
from sklearn.multiclass import OneVsOneClassifier


scores = []
Cs = list(map(int, range(1, 100, 1)))

for c in tqdm(Cs):
    clf = OneVsOneClassifier(svm.SVC(C=c))
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))

plot_score(Cs, scores, 'C-argument')

In [ ]:
scores = []

for c in tqdm(Cs):
    clf = OneVsOneClassifier(svm.SVC(C=c))
    cvs = cross_val_score(clf, x, y, cv=5)
    scores.append(cvs.mean())
plot_score(Cs, scores, 'C-argument')

##### форма функции решения: one vs rest

In [ ]:
scores = []

for c in tqdm(Cs):
    svc = svm.SVC(C=c, break_ties=True)
    svc.fit(X_train, y_train)
    scores.append(svc.score(X_test, y_test))

plot_score(Cs, scores, 'C-argument')

In [ ]:
from sklearn.multiclass import OneVsRestClassifier


scores = []
for c in tqdm(Cs):
    clf = OneVsRestClassifier(svm.SVC(C=c))
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))

plot_score(Cs, scores, 'C-argument')

In [ ]:
scores = []
Cs = list(map(int, range(1, 50, 1)))
for c in tqdm(Cs):
    clf = OneVsRestClassifier(svm.SVC(C=c))
    cvs = cross_val_score(clf, x, y, cv=5)
    scores.append(cvs.mean())

plot_score(Cs, scores, 'C-argument')

#### $\nu$-svc

In [ ]:
scores = []
Nus = np.arange(0.01, 0.5, 0.01)
for nu in tqdm(Nus):
    nu_svc = svm.NuSVC(nu=nu, decision_function_shape='ovo')
    nu_svc.fit(X_train, y_train)
    scores.append(nu_svc.score(X_test, y_test))

plot_score(Nus, scores, 'Nu-argument')

### Нейронные сети

#### Базовая модель

Возьмем в качестве модели многослойный перцептрон, для борьбы с переобучением
воспользуемся слоями `Dropout`. Кроме того, добавим между слоями
нормализацию по подвыборке для того, чтобы сгладить процесс обучения.

В качестве функции активации выберем `leaky_relu`.

In [ ]:
from keras.layers import Input, Dropout, Dense, BatchNormalization

clear_session()

lr = 0.001

model = keras.Sequential()
model.add(Input(X_train.shape[1]))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.3))

model.add(Dense(128, activation='leaky_relu'))
model.add(Dropout(.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='leaky_relu'))
model.add(Dense(8, activation='softmax'))


model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

In [ ]:
np.unique(y_train)

In [ ]:
nn_y_train = np.array(list(map(label2vec(n_classes), y_train)))
nn_y_test = np.array(list(map(label2vec(n_classes), y_test)))
nn_y_valid = np.array(list(map(label2vec(n_classes), y_valid)))

In [ ]:
run = input()

callbacks = [
  TensorBoard(),
  ModelCheckpoint(f'{run}/checkpoint/', save_best_only=True, save_weights_only=True, monitor='categorical_accuracy', verbose=1),
  CSVLogger("logs.csv")
]

In [ ]:
model.fit(X_train, nn_y_train, validation_data=(X_valid, nn_y_valid), epochs=200, callbacks=callbacks, batch_size=128)

In [ ]:
model.load_weights(f'{run}/checkpoint/')
model.evaluate(X_test, nn_y_test)

In [ ]:
models.append((model, 'keras'))

### Сравнение моделей, обученных на мета-данных

In [ ]:
for entry in models:
  model_type = entry[1]
  model = entry[0]
  if model_type == 'sklearn':
    model.fit(X_train, y_train)
    results = model.score(X_test, y_test)
  else:
    results = model.evaluate(X_test, nn_y_test)
  print(f'Результат для {model.__class__.__name__}: {results}')

### Сверточная нейронная сеть

In [ ]:
from loaders import AudioDataset

from torch.utils.data import DataLoader

In [ ]:
train = pd.DataFrame(columns=selected.columns)
valid = pd.DataFrame(columns=selected.columns)
test = pd.DataFrame(columns=selected.columns)

train_size = 0.8
valid_size = 0.1

for i in range(0, 8):
  cur = selected[selected['track_genres'] == i]
  
  n = len(cur)
  train = pd.concat([train, cur.iloc[:int(train_size * n)]])
  valid = pd.concat([valid, cur.iloc[int(train_size * n):int((train_size + valid_size) * n)]])
  test = pd.concat([test, cur.iloc[int((train_size + valid_size) * n):]])


In [ ]:
test_dataset = AudioDataset(test, suffix='test',
                            sr=44100, win_length=1380, 
                            hop_length=345, data_dir=DATA_DIR)
val_dataset = AudioDataset(valid, suffix='val',
                           sr=44100, win_length=1380, 
                           hop_length=345, data_dir=DATA_DIR)


In [ ]:
train_dataset = AudioDataset(train, suffix='train', 
                             sr=44100, win_length=1380, 
                             hop_length=345, data_dir=DATA_DIR)

In [ ]:
batch_size = 128

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
train_dataset.unload()

In [ ]:
from torchvision.models import resnet18

model = resnet18()

device = torch.device("mps")

torch.manual_seed(69)

for param in model.parameters():
  param.requires_grad = False
  
model.fc = torch.nn.Sequential(
  torch.nn.Dropout(p=0.2, inplace=True),
  torch.nn.Linear(in_features=512,
                  out_features=256,
                  bias=True),
  torch.nn.Dropout(p=0.2, inplace=True),
  torch.nn.Linear(in_features=256,
                  out_features=8,
                  bias=True))

ts.summary(model, (3, 640, 480))


In [ ]:
from utils import save_state, plot_csv, restore_state

model, init_epoch, best_loss, losses = restore_state(model, 'checkpoint/checkpoint.pt')
model.to(device)
crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)
model.train()

epochs = 100

n_batches = len(train_loader)

for epoch in range(init_epoch, epochs):
  print(f'Epoch {epoch + 1}/{epochs}')
  pbar = tf.keras.utils.Progbar(target=n_batches)
  
  correct = 0
  samples = 0
  for i, (inputs, targets) in enumerate(train_loader):
    inputs = inputs.to(device)
    
    targets = targets.argmax(dim=1)
    targets = targets.to(device)
    
    opt.zero_grad()
    outputs = model(inputs)
    
    train_loss = crit(outputs, targets)
    train_loss.backward()
    
    correct += int(torch.sum(outputs.argmax(dim=1) == targets))
    samples += len(targets)
    
    opt.step()
    pbar.update(i, values=[("loss", train_loss.item()), ("acc", correct / samples)])
    train_loader.dataset.unload()
      
  model.eval()
  
  with torch.no_grad():
    correct = 0
    samples = 0
    for inputs, targets in val_loader:
      inputs = inputs.to(device)
      
      targets = targets.argmax(dim=1)
      targets = targets.to(device)
      
      outputs = model(inputs)
      val_loss = crit(outputs, targets)
      
      correct += int(torch.sum(outputs.argmax(dim=1) == targets))
      samples += len(targets)
      
  
  losses['epoch'].append(epoch)
  losses['loss_type'].append('train')
  losses['loss'].append(train_loss.item())
  
  losses['epoch'].append(epoch)
  losses['loss_type'].append('val')
  losses['loss'].append(val_loss.item())
  
  save_state(model, epoch, losses, best_loss)
   
  if val_loss.item() < best_loss:
    best_loss = val_loss.item()
    torch.save(model.state_dict(), 'best.pt')
  
  plot_csv('checkpoint/losses.csv')
      
  pbar.update(n_batches, values=[("val_loss", val_loss.item()), ("val_acc", correct / samples)])

In [ ]:
sns.lineplot(data=losses, x='epoch', y='loss', hue='loss_type')

In [ ]:
torch.load('best.pt')

In [ ]:
torch.save(model.state_dict(), "output")